In [117]:
import secrets
from operator import xor
from hashlib import sha256

NUM_LABEL_BITS = 128

class Label(object):
    def __init__(self, label, bit):
        self.label = label
        self.bit = bit

    @classmethod
    def new(cls, bit):
        label = secrets.randbits(NUM_LABEL_BITS)
        return cls(label, bit)

    @classmethod
    def new_delta(cls):
        label = secrets.randbits(NUM_LABEL_BITS) | 1
        return cls(label, 0)
        
    def selector_bit(self):
        return self.label & 1

    def negate(self, delta):
        label = xor(self.label, delta.label)
        bit = xor(self.bit, 1)

        return self.__class__(label, bit)

    def xor(self, other):
        label = xor(self.label, other.label)
        bit = xor(self.bit, other.bit)

        return self.__class__(label, bit)

    def mul(self, bit):
        assert((bit == 0) or (bit == 1))
        if bit == 0:
            return self.__class__(0, 0)
        else:
            return self

    def hash(self):
        # Convert integer to bytes (big-endian)
        x_bytes = int(self.label).to_bytes(16, byteorder='big', signed=False)
        # Compute SHA-256 hash
        hash_bytes = sha256(x_bytes).digest()
        # Truncate to 128 bits (first 16 bytes)
        label = int.from_bytes(hash_bytes[:16], byteorder='big')

        return self.__class__(label, 0)

    def is_zero(self):
        return self.label == 0

class Wire(object):
    ## Global and secret offset for all wires
    DELTA = Label.new_delta()
    __slots__ = ('name', 'label0')  # Prevent dynamic attributes
    
    def __init__(self, name):
        self.name = name
        self.label0 = Label.new(0)

    def permutate_bit(self):
        return xor(self.label0.selector_bit(), self.label0.bit)

    def label1(self):
        return self.label0.negate(Wire.DELTA)

    def select_with_permutate_bit(self):
        if self.permutate_bit() == 0:
            return self.label0
        else:
            return self.label1()

    def select(self, var):
        if var == 0:
            return self.label0
        else:
            return self.label1()

    @classmethod
    def new(cls, name, label0):
        return cls(name, label0)

class Gate(object):
    def __init__(self, name, alphas, input_wires):
        assert(len(alphas) == 3)
        self.name = name
        self.alpha_A = alphas[0]
        self.alpha_B = alphas[1]
        self.alpha_C = alphas[2]
        assert(len(input_wires) == 2)
        self.wire_A = input_wires[0]
        self.wire_B = input_wires[1]
        
################################### Setup phase, generating all input labels
w_A, w_B = Wire("W_A"), Wire("W_B")
for w in [w_A, w_B]:
    assert(w.permutate_bit() == xor(w.label0.selector_bit(), w.label0.bit))
    assert(w.permutate_bit() == xor(w.label1().selector_bit(), w.label1().bit))

print("permutate bit: p_A = {0}, p_B = {1}".format(w_A.permutate_bit(), w_B.permutate_bit()))
print("selector bit: s_A^0 = {0}, s_A^1 = {1}, s_B^0 = {2}, s_B^1 = {3}".format(
    w_A.label0.selector_bit(), w_A.label1().selector_bit(), w_B.label0.selector_bit(), w_B.label1().selector_bit())
     )
print("\n")
    
## --------------------------------------------------------- ##
# Garbler half-gate:
# f_G(a, p_B) = (a \xor \alpha_a) \land (p_B \xor \alpha_b) \xor \alpha_c
#
# before GRR:
# e_0 = H(W_A^0) \xor f_G(0, p_B) R \xor W_{C_G}^0
# e_1 = H(W_A^1) \xor f_G(1, p_B) R \xor W_{C_G}^0
# T_G = Permutate([e_0, e_1]) with p_A
#
# Evaluator half-gate:
# f_E(a, b \xor p_B) = (a \xor \alpha_a) \land (b \xor p_B)
# 
# before GRR:
# e_0 = H(W_B^{p_B}) \xor W_{C_E}^0
# e_1 = H(W_B^{p_B \xor 1}) \xor W_{C_E}^0 \xor W_A^\alpha_a
## --------------------------------------------------------- ##

def f_G(a, p_B, alpha_A, alpha_B, alpha_C):
    return xor(xor(a, alpha_A) & xor(p_B, alpha_B), alpha_C)

################################### General Gate with odd-ones in Yao's garbling table
alphas = [[0, 0, 0], [1, 1, 1], [0, 0, 1], [1, 1, 0]]
operators = ["AND", "OR", "NAND", "NOR"]
p_A = w_A.permutate_bit()
p_B = w_B.permutate_bit()

for (alpha, operator) in zip(alphas, operators):
    ########################################## Garbling
    ## garbler half-gate
    w_CG_0 = w_A.select_with_permutate_bit().hash().xor(Wire.DELTA.mul(f_G(p_A, p_B, alpha[0], alpha[1], alpha[2])))
    T_G = [w_A.label0.hash().xor(w_A.label1().hash()).xor(Wire.DELTA.mul(xor(p_B, alpha[1])))]
    ## evaluator half-gate
    w_CE_0 = w_B.select_with_permutate_bit().hash()
    T_E = [w_B.label0.hash().xor(w_B.label1().hash()).xor(w_A.select(alpha[0]))]

    ########################################## Evaluating
    for a, b in [(0, 0), (0, 1), (1, 0), (1, 1)]:
        w_A_a, w_B_b = w_A.select(a), w_B.select(b)
        s_A_a, s_B_b = w_A_a.selector_bit(), w_B_b.selector_bit()

        ## Garbler Half-gate
        if s_A_a == 0:
            w_CG_c = w_A_a.hash()
        else:
            w_CG_c = T_G[0].xor(w_A_a.hash())
        CG_c = 0 if (w_CG_c.label == w_CG_0.label) else 1

        ## Evaluator Half-gate
        if s_B_b == 0:
            w_CE_c = w_B_b.hash()
        else:
            w_CE_c = T_E[0].xor(w_B_b.hash()).xor(w_A_a)
        CE_c = 0 if (w_CE_c.label == w_CE_0.label) else 1

        ## Output label
        w_c = w_CG_c.xor(w_CE_c)

        ## Check
        c_bit =  xor(CG_c, CE_c)
        if operator == "AND":
            true_c_bit = a & b
        elif operator == "OR":
            true_c_bit = a | b
        elif operator == "NAND":
            true_c_bit = not (a & b)
        elif operator == "NOR":
            true_c_bit = not (a | b)
        assert(true_c_bit == c_bit)
        print("{0} {1} {2} = {3} = {4} xor {5} = {6}".format(a, operator, b, true_c_bit, CG_c, CE_c, c_bit))
    print("\n")

permutate bit: p_A = 0, p_B = 0
selector bit: s_A^0 = 0, s_A^1 = 1, s_B^0 = 0, s_B^1 = 1


0 AND 0 = 0 = 0 xor 0 = 0
0 AND 1 = 0 = 0 xor 0 = 0
1 AND 0 = 0 = 0 xor 0 = 0
1 AND 1 = 1 = 0 xor 1 = 1


0 OR 0 = 0 = 0 xor 0 = 0
0 OR 1 = 1 = 0 xor 1 = 1
1 OR 0 = 1 = 1 xor 0 = 1
1 OR 1 = 1 = 1 xor 0 = 1


0 NAND 0 = True = 1 xor 0 = 1
0 NAND 1 = True = 1 xor 0 = 1
1 NAND 0 = True = 1 xor 0 = 1
1 NAND 1 = False = 1 xor 1 = 0


0 NOR 0 = True = 1 xor 0 = 1
0 NOR 1 = False = 1 xor 1 = 0
1 NOR 0 = False = 0 xor 0 = 0
1 NOR 1 = False = 0 xor 0 = 0


